In [1]:
!uv sync
%load_ext autoreload
%autoreload 2

Resolved 153 packages in 6ms
Audited 150 packages in 0.52ms


In [2]:
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import data_exploration.weather_stations as ws
import data_exploration.viz as viz

In [3]:
folder = pathlib.Path('../data/station_data_parquet/')
flist = list(folder.glob('*.parquet'))
df = pd.concat([ws.read_parquet(f) for f in flist[10:50]], axis=0)

2025-03-12 12:45:09.477 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU3786-1-1.parquet
2025-03-12 12:45:09.545 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU1523-4-1.parquet
2025-03-12 12:45:09.561 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU371-A3-1.parquet
2025-03-12 12:45:09.576 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFM00068490.parquet
2025-03-12 12:45:09.590 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFU184-A3-1.parquet
2025-03-12 12:45:09.603 | DEBUG    | data_exploration.weather_stations:read_parquet:16 - Skipping file - too few data ../data/station_data_parquet/SFM00068848.parque

Gridded temperature - doing naive interpolation not realistic
Data has lots of missing spatial data 
How does data compare with ERA5 - can we 

1. Filter stations by those that have temperature
2. Remove 0°C - likely bad measurment
3. Remove outliers (IQR * 3) - seasonal option also possible


In [4]:
grp = df.groupby('Station_name', as_index=False)
i0 = [grp.groups[g][0] for g in grp.groups]
n_obs = {g: len(grp.groups[g]) for g in grp.groups}

df_locs = (
    df.loc[i0]
    .reset_index()
    .set_index('Station_name')
    .assign(n_obs=n_obs)
    [['datetime', 'n_obs', 'Latitude', 'Longitude', 'geometry']])

In [36]:
fig_refs = {}
fig_ref_fmt = "<img src='{fig_path}' width='800px'>".replace('\'', '\"')
# fig_ref_fmt = "<iframe src='{fig_path}'>"
for station in df_locs.index:
    figname = viz.plot_station_timeseries(df, station, save_dir="../docs/figures/")
    fig_refs[station] = fig_ref_fmt.format(fig_path=figname).replace('../docs/', '')

df_locs['fig_ref'] = pd.Series(fig_refs)

In [43]:
import folium

map = folium.Map([-29, 26], tiles=None, zoom_start=6)

folium.TileLayer(tiles='openstreetmap', name='OpenStreetMap').add_to(map)
folium.TileLayer(**viz.GOOGLE_SATELLITE, name='Google Satellite').add_to(map)
folium.TileLayer(**viz.GOOGLE_TERRAIN, name='Google Terrain').add_to(map)

tootlip_cols = ['name', 'n_obs', 'Latitude', 'Longitude', 'datetime']
tootlip_classes = 'table table-striped table-hover table-condensed table-responsive'

for name, row in df_locs.iterrows():
    row['name'] = name
    tooltip = row[tootlip_cols].to_frame().to_html(classes=tootlip_classes)

    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=3,
        color='red',
        popup=folium.Popup(row['fig_ref']),
        tooltip=tooltip,
    ).add_to(map)

# map.add_css_link('custom_css', '../notebooks/custom_folium.css')
folium.LayerControl(collapsed=False).add_to(map)
map.save('../docs/index.html')